In [1]:
# import libraries
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

# following imports are required by PKI
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [2]:
class Client:
   def __init__(self):
      random = Crypto.Random.new().read
      self._private_key = RSA.generate(1024, random)
      self._public_key = self._private_key.publickey()
      self._signer = PKCS1_v1_5.new(self._private_key)

   @property
   def identity(self):
      return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [3]:
# How to use the client
michael = Client()
print(michael.identity)

30819f300d06092a864886f70d010101050003818d0030818902818100958a67e5dc4ba4507cffc6485aeb5c46fdfff7048d1b31f32f36b46a13ef773258e76498e3139925f7dd45148ddf7f78daf43feb5c0fbc963f1f95f05b7f6d1e95f1ecc226b01d7679c258617d05e1f80252fdc4aaa206d45c50a5d24b76f51f983c0f2526f6de0ca5a0d7045671bda37306a8ec378c0e91b0389c1c0960fde30203010001


In [4]:
class Transaction:
    '''
    A class to represent a transaction

    ...
    
    Attributes
    -----------
    sender : str
        The senders public key taken from their instance of the Client()
    recipient : str
        The recipients public key taken from their instance of the Client()
    value : int
        The amount of coins to be sent in the transaction
    time : datetime
        Time the transaction took place

    Mehtods
    -------
    to_dict():
        Utiltiy method that combines all four attributes into a dictionary object
    sign_transaction():
        Used to sign the dictionary object from to_dict with the private key of the sender
    '''
    def __init__(self, sender, recipient, value):
        '''
        Constructs the 4 needed attributes for the person object

            Parameters:
            -----------
                sender : str
                    The senders public key taken from their instance of the Client()
                recipient : str
                    The recipients public key taken from their instance of the Client()
                value : int
                    The amount of coins to be sent in the transaction
        
            Returns:
            --------
                instance variables that can be used by other methods.
        '''
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        '''
        Utiltiy method that combines all four attributes into a dictionary object

        The if, else checks whehter the sender of the transaction has the identity of Genesis. 
        If so they are assigned the value of genesis so their identity is kept secret

        Returns
        -------
        Dictionary Object containing the four attributes that make up a transaction
        '''
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time})

     # Used to retrieve the senders private key       
    def sign_transaction(self):
        '''
        Used to sign the dictionary object from to_dict with the private key of the sender

        Returns
        -------
        The private key of the sender
        '''
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')


In [5]:
# Testing the Transaction Class

# Michael is going to send 5 coins to Tyler
# To do this we first create the clients
michael = Client()
tyler = Client()

# Create the transaction with the following code
t = Transaction(
    michael,  # Sender
    tyler.identity, # Gets tylers public key
    5.0  # Amount transferred
)

# Printing the transaction to review it
print(t.to_dict())

# Once we have created the transaction we can sign it with the senders private key.
signature = t.sign_transaction()
print(signature)


OrderedDict([('sender', '30819f300d06092a864886f70d010101050003818d0030818902818100cecc5a8665dd46424f0b1fc225e89d2a3d4f81e7aabf32dc15704d94dc236c42a689937ca519678194623452ca3af6bde616d80e38aea1f309c4a00d291dd3fa543c7aad85dc27ae2839400e043590c13a7632ace2eb9ae7b2b9f24fce3f358f9e3487fd0d2cb5be0fd33ee88b6914617c58b59956fe08d47e4a14dd79d1021d0203010001'), ('recipient', '30819f300d06092a864886f70d010101050003818d0030818902818100cb68d12ede8d6400c552bd36640641f055282946adf82f290f621c6347cce76da9808b37660232d4252a135fae258263ea6828aabff1dcd35630876687394b583a4ae7e7a6657cf0c79e2653885e5fe6c9c2261f226274ee6a998141a6d3f9eb533b870b65150285954b008ffc9dd7ee21118fbc9b52e46eb416c2998058b04f0203010001'), ('value', 5.0), ('time', datetime.datetime(2021, 2, 1, 17, 38, 44, 481989))])
68219d5263729ece4f0831ce1fccd94e870b05907526ea8241d6ab614fbca85ff9e2807fcbc182bec7977e4a0f8362a5fbbb119e750b84cb9c0f2f4ed70d76a56eca354136ef6f92e00445dbe8b78af7f667e7a744c762cb3aae57cfdf52a358a4e5a62b93f553cdc9c36f9d3f4c469a19

In [6]:
# Next section to start https://www.tutorialspoint.com/python_blockchain/python_blockchain_creating_multiple_transactions.htm

In [7]:
def display_transaction(transaction):
   #for transaction in transactions:
   dict = transaction.to_dict()
   print ("sender: " + dict['sender'])
   print ('-----')
   print ("recipient: " + dict['recipient'])
   print ('-----')
   print ("value: " + str(dict['value']))
   print ('-----')
   print ("time: " + str(dict['time']))
   print ('-----')

In [8]:
# Queue Where transactions are stored before they are added to the blockchain
# Not implementing the queue management logic
transactions = []

In [9]:
# Creating multiple clients
mike = Client()
steve = Client()
rose = Client()
rachel = Client()

In [10]:
# Initiating the first transaction
t1 = Transaction(
    mike,
    steve.identity,
    15.0
)

In [11]:
# Adding the first transaction to the queue
t1.sign_transaction()
transactions.append(t1)

In [12]:
t2 = Transaction(
   mike,
   rose.identity,
   6.0
)
t2.sign_transaction()
transactions.append(t2)
t3 = Transaction(
   steve,
   rachel.identity,
   2.0
)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction(
   rose,
   steve.identity,
   4.0
)
t4.sign_transaction()
transactions.append(t4)
t5 = Transaction(
   mike,
   rose.identity,
   7.0
)
t5.sign_transaction()
transactions.append(t5)
t6 = Transaction(
   rose,
   rachel.identity,
   3.0
)
t6.sign_transaction()
transactions.append(t6)
t7 = Transaction(
   rachel,
   steve.identity,
   8.0
)
t7.sign_transaction()
transactions.append(t7)
t8 = Transaction(
   rachel,
   mike.identity,
   1.0
)
t8.sign_transaction()
transactions.append(t8)
t9 = Transaction(
   steve,
   mike.identity,
   5.0
)
t9.sign_transaction()
transactions.append(t9)
t10 = Transaction(
   steve,
   rose.identity,
   3.0
)
t10.sign_transaction()
transactions.append(t10)

In [13]:
for transaction in transactions:
    display_transaction (transaction)
    print('----------------')

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b10c998544463e3fae38f898518bdf4c383c033db77dccec1905ba3f94ab371545c5fbb43bf04e096e306d731146b42bb7359234f4bccadfdbb20487e52a123cff037988d89d6dc906f228648eeb635577c66c3e4cd11a958cbce84f11908787a63a024a70847b383e555b5260bf455df1c67cddb8b875d915ddb5daf72f9f870203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100d62bc4caf8049f1a9ecaba11f75d51954747707339e25f00ba146761cd7d54b62c7063ad481ce8d1dff4c12ed8606fdd34c6c215a174b0c945f191831b068fd88774a398bf3606f508716a0394d2d1d280062b7562362f64af07fbabb590aeb2369c5c9ff919ac065b12e5d8aca8d7d3388d949c3108b4f9b280272667a4e4470203010001
-----
value: 15.0
-----
time: 2021-02-01 17:38:45.258251
-----
----------------
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b10c998544463e3fae38f898518bdf4c383c033db77dccec1905ba3f94ab371545c5fbb43bf04e096e306d731146b42bb7359234f4bccadfdbb20487e52a123cff037988d89d6dc906f228648eeb635577c66c3e4cd11a958cb

A block consists of a varying number of transactions. For simplicity, in our case we will assume that the block consists of a fixed number of transactions

In [14]:
# blockchain is a list of blocks chained to each other
# For our list we will create the following list variable
TPCoins = []

class Block:
    '''
    Blockchain
    ...

    Attributes
    -----------
    verified_transactions : list
        Verified transactions will be added to this block. Each block also contains the
        hash value of the previous block. Making the chain immutable
    previous_block_hash : str
        Instance varaible to store the previous hash
    Nonce : str
        Variable to store the Nonce created by the miner during the mining process

    Methods
    -------
    dump_blockchain:
        dumps the contents of the entire blockchain.
    '''
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ''
        self.Nonce = ''

    def dump_blockchain(self):
        '''
        Dumps the contents of the entire blockchain.

        Description
        ------------
        1. It prints the number of blocks in the current blockchain
        2. Next the blocks are copied to a temp variable named block_temp
        3. Print block number as heading for each block
        4. Then iterate over the transactions in each block
        '''
        print('Number of blocks in the chain: ' + str(len (self)))

        for x in range (len(TPCoins)):
            block_temp = TPCoins[x]
            print('block # ' + str(x))
            for transaction in block_temp.verified_transactions:
                display_transaction (transaction)
                print ('--------------')
            print ('=====================================')

# Global variable 
last_block_hash = ''

In [15]:
# Genesis block: 500 coins are given out to the client michael
michael = Client()

t0 = Transaction (
    'Genesis',
    michael.identity,
    500.00
)

# Create an instance of the Block class
block0 = Block()

# Where this is the first block we initialize the previous_block_hash and Nonce to none.
block0.previous_block_hash = None 
block0.Nonce = None

# Add the transaction to the verified transactions
block0.verified_transactions.append (t0)

# The block is now ready to be added to the blockchain. Before we do this we will hash the
# block and store its value in the global variable `last_block_hash`
digest = hash (block0)
last_block_hash = digest

In [ ]:



# https://www.tutorialspoint.com/python_blockchain/python_creating_blockchain.htm
